In [ ]:
#import moduel
import numpy as np
import copy, math
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
np.random.seed(2290) #400132290

In [ ]:
#generate data
def func(x):
    return np.sin(4 * np.pi * x + np.pi/2)

def generate_data(p):    
    #generate sets
    X_train = np.linspace(0.,1.,p)
    X_valid = np.linspace(0.,1.,p*10)
    #this is the label y, this this case it is t = sin(4*pi*x)
    # t_train = func(X_train) + 0.3*np.random.randn(12)
    # t_valid = func(X_valid) + 0.3*np.random.randn(120)

    mean = 0
    variance = 0.0625
    # noise = np.random.normal(mean, np.sqrt(variance), size=(10,))
    t_train = func(X_train) + np.random.normal(mean, np.sqrt(variance), size=(p,))
    t_valid = func(X_valid) + np.random.normal(mean, np.sqrt(variance), size=(p*10,))
    
    return X_train,X_valid,t_train,t_valid

def generate_matrix(x_train,t_train,x_valid,t_valid,exp):
    xx_train = x_train[:,np.newaxis] ** [i for i in range(exp+1)]
    xx_valid = x_valid[:,np.newaxis] ** [i for i in range(exp+1)]

    tt_train = t_train[:,np.newaxis]
    tt_valid = t_valid[:,np.newaxis]
    # tt_train = np.array(tt_train)
    # tt_valid = np.array(tt_valid)

    return xx_train,xx_valid,tt_train,tt_valid

#def apply_scaling():
    

In [ ]:
#training model
def train(X, t_train):
    #w = (x^T * x)^-1 * (x^T * t)
    w = np.linalg.inv(X.T @ X) @ (X.T @ t_train)
    return w

def train_regulation(X,t_train,B):
    w = np.linalg.inv(X.T @ X + B) @ (X.T @ t_train)
    return w

def pred(X, w):
    pred = (w * X).sum(axis=1, keepdims=True)
    return pred

In [ ]:
#calc error
def calc_error(w,x_train,t_train,x_valid,t_valid):
    # pred : w*x = y
    # (y - t)^T (y - t) N
    training_error = np.sum(np.square( (pred(x_train,w)- t_train) )) / t_train.shape[0]
    valid_error = np.sum(np.square( (pred(x_valid,w)- t_valid) )) / t_valid.shape[0]

    return training_error, valid_error

def calc_error_regulation(w, x_train, t_train, x_valid, t_valid, lamda):
    training_error, valid_error = calc_error(w, x_train, t_train, x_valid, t_valid)
    training_error += lamda * np.sum(np.square(w))
    valid_error += lamda * np.sum(np.square(w))

    return training_error,valid_error


In [ ]:
#plot
def plot_data(idx, t_pred, x_train, t_train, x_valid, t_valid, reg=False):
    # create a new figure per model
    fig = plt.figure(idx)
    fig.suptitle(f"{'Lambda=' if reg else 'Degree '}{idx}")

    # plot the training and validation data
    plt.plot(x_train, t_train, '.', color="blue", label="Training data", mfc="none")
    plt.plot(x_valid, t_valid, '.', color="red", label="Validation data", mfc="none")

    # plot the model function and the true function
    plt.plot(x_valid, func(x_valid), color="black", label="f_true")
    plt.plot(x_valid, t_pred, color="lightgreen", label="f_pred")

    # label the axies
    plt.xlabel("x")
    plt.ylabel("t")
    
    # show the figure
    plt.legend(loc="best")
    plt.show(block=False)

def plot_error_curves(x_train, t_train, t_valid):
    # create a new figure per model
    fig = plt.figure()
    fig.suptitle(f"Errors")

    # plot the training and validation data
    plt.plot(x_train, t_train, '-', color="blue", label="Training data", mfc="none")
    plt.plot(x_train, t_valid, '-', color="red", label="Validation data", mfc="none")

    # show the figure
    plt.legend(loc="best")
    plt.show(block=False)

In [ ]:
p = 12
x_train,x_valid,t_train,t_valid = generate_data(p)
ans = {
    "M": [ i for i in range(0,p)],
    "training_error" : [],
    "valid_error" : []
}


for exp in ans["M"]:

    xx_train,xx_valid,tt_train,tt_valid = generate_matrix(x_train,t_train,x_valid,t_valid,exp)
    # print(xx_train.shape)
    # print(xx_valid.shape)
    # print(tt_train.shape)
    # print(tt_valid.shape)
    #print(xx_train)
    w = train(xx_train,t_train)
    print(w)
    train_error, valid_error = calc_error(w,xx_train,tt_train,xx_valid,tt_valid)
    print(f"Degree {exp}: training error = {train_error:22}, validation error = {valid_error}")
    ans["training_error"].append(train_error)
    ans["valid_error"].append(valid_error)
    plot_data(exp, pred(w, xx_valid), x_train, t_train, x_valid, t_valid)

print(f"The lowest training error was at degree {ans['M'][np.argmin(ans['training_error'])]} and the "
          f"lowest validation error was at degree {ans['M'][np.argmin(ans['valid_error'])]}")

print(xx_train[-1,:])
#apply regulation
xx_train = np.delete(xx_train,0,1)
xx_valid = np.delete(xx_valid,0,1)

sc = StandardScaler()
xx_train = sc.fit_transform(xx_train)
xx_valid = sc.transform(xx_valid)
xx_train = np.c_[np.ones(t_train.shape[0]), xx_train]
xx_valid = np.c_[np.ones(t_valid.shape[0]), xx_valid]
regulation = {
    "lamda" : np.exp([i for i in range(-100,-1)]).tolist(),
    "training_error" : [],
    "valid_error" : [],
    "w" : []
}

for lamda in regulation["lamda"]:
    #create lambda
    B = np.zeros((p,p))
    np.fill_diagonal(B, lamda * t_train.shape[0])

    w = train_regulation(xx_train,t_train,B)
    regulation["w"].append(w)
    train_error, valid_error = calc_error_regulation(w,xx_train,tt_train,xx_valid,tt_valid,lamda)
    
    #print(f"lamda: {lamda}: training error = {train_error:22}, validation error = {valid_error}")
    regulation["training_error"].append(train_error) 
    regulation["valid_error"].append(valid_error)

lowest_valid_error = np.argmin(regulation["valid_error"])

lambda1 = regulation['lamda'][lowest_valid_error]
lambda2 = regulation['lamda'][-5]

print(f"The lowest regularised validation error was at constant {lambda1}")

plot_data(lambda1, pred(regulation["w"][lowest_valid_error], xx_valid), x_train, t_train, x_valid,
              t_valid, reg=True)
plot_data(lambda2, pred(regulation["w"][-4], xx_valid), x_train, t_train, x_valid, t_valid,
              reg=True)

plot_error_curves(ans["M"],ans["training_error"],ans["valid_error"])
print(ans["training_error"])
